# Install

In [1]:
import os
os.chdir('./GymGo-master')

In [2]:
! pip install -e .

Obtaining file:///Users/yuetongliu/Desktop/Reinforcement_Learning_on_Go/Code/14_02_24_Go/GymGo-master
  Attempting uninstall: gym-go
    Found existing installation: gym-go 0.0.1
    Uninstalling gym-go-0.0.1:
      Successfully uninstalled gym-go-0.0.1
  Running setup.py develop for gym-go


In [16]:
import gym
import random
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [110]:
size = 9
go_env = gym.make('gym_go:go-v0', size=size, reward_method = "heuristic")
ept = go_env.state()[0]

# define state value function as a dict
# key is position of b and w
# value is state value (a number)
value_function = dict()

# add new state if not exist
if tuple(go_env.state()[0:2].reshape(size**2*2)) not in value_function:
    value_function[tuple(go_env.state()[0:2].reshape(size**2*2))] = 0


# define action value function as a dict
# key is position of b and w
# value is action value (an array)
action_function = dict()
if tuple(go_env.state()[0:2].reshape(size**2*2)) not in action_function:
    action_function[tuple(go_env.state()[0:2].reshape(size**2*2))] = np.zeros(size**2+1)

In [83]:
def apply_policy(state, epsilon):
        """
        Apply the policy of the agent
        Args:
            state: tuple of length 2
            epsilon: exploration probability, 0 for greedy behavior, 1 for pure exploration
        Returns:
            the selected action for the state under the current policy
        """
        greedy_action_value = np.max(action_function[tuple(state[0:2].reshape(size**2*2))])
        greedy_indices = [i for i, a in enumerate(action_function[tuple(state[0:2].reshape(size**2*2))]) if
                          a == greedy_action_value]
        action_index = np.random.choice(greedy_indices)
        if np.random.uniform(0, 1) < epsilon:
            action_index = go_env.uniform_random_action()
        return action_index

In [111]:
state = go_env.state()
apply_policy(state, 0.9)

54

In [119]:
def sarsa_td(n_episodes=100, alpha=0.01, gamma=0.9):
        """
        Run the sarsa control algorithm (TD0), finding the optimal policy and action function
        :param n_episodes: int, amount of episodes to train
        :param alpha: learning rate
        :param gamma: discount factor of future rewards
        :return: finds the optimal policy for move chess
        """
        
        for k in range(n_episodes):
            go_env = gym.make('gym_go:go-v0', size=size, reward_method = "heuristic")
            state = go_env.state()
            
            done = False
            epsilon = max(1 / (1 + k), 0.05)
            
            # 
            if tuple(go_env.state()[0:2].reshape(size**2*2)) not in action_function:
                action_function[tuple(go_env.state()[0:2].reshape(size**2*2))] = np.zeros(size**2+1)
            
            while not done:
                # make action
                # state = self.env.state
                action_index = apply_policy(state, epsilon)
                
                successor_state, reward, done, info = go_env.step(action_index)
                #reward, episode_end = self.env.step(action)
                
                # action value
                if tuple(state[0:2].reshape(size**2*2)) not in action_function:
                    action_function[tuple(state[0:2].reshape(size**2*2))] = np.zeros(size**2+1)
                    action_value = 0
                else:
                    action_value = action_function[tuple(state[0:2].reshape(size**2*2))][action_index]
                
                # update action value function
                # success 
                if tuple(successor_state[0:2].reshape(size**2*2)) not in action_function:
                    action_function[tuple(successor_state[0:2].reshape(size**2*2))] = np.zeros(size**2+1)
                    
                #successor_state = self.env.state
                successor_action_index = apply_policy(successor_state, epsilon)

                successor_action_value = action_function[tuple(state[0:2].reshape(size**2*2))][successor_action_index]
                    
                

                q_update = alpha * (reward + gamma * successor_action_value - action_value)

            
                action_function[tuple(state[0:2].reshape(size**2*2))][action_index] += q_update
                #self.agent.policy = self.agent.action_function.copy()

In [120]:
sarsa_td()

AssertionError: ('Invalid move', (7, 4))

In [78]:
# 1000 eposides
alpha=0.05
lamb=0.9
for i in range(10000): 
    
        go_env = gym.make('gym_go:go-v0', size=size, reward_method = "heuristic")
    
        done = False
        reward = 0
        for i in range(-size**2,size**2+1):
            value_function[i].append(0)
            
        while not done:
            action = go_env.uniform_random_action()
            suc_state, suc_reward, done, info = go_env.step(action)

            value_function[reward].append(value_function[reward][-1] + alpha * (
                suc_reward + lamb * value_function[suc_reward][-1] - value_function[reward][-1])) 

            reward = suc_reward
            if go_env.game_ended():
                break
            

        # white    
            action = go_env.uniform_random_action()
            state_w, reward_w, done, info = go_env.step(action)
        
    
        
    
